In [324]:
import numpy as np
import collections

class Unit:
    def __init__(self, char, tile):
        self.hp    = 200        
        self.type  = char
        if char == 'E':
            self.atk = 14 #part1 = 3, part2 = 14
        else:
            self.atk = 3
        self.moved = False
        self.tile  = tile
    
    def getTargets(self):
        return units[[u.type != self.type and u.hp > 0 for u in units]]
    
    def attack(self, index):
        y, x = index
        targets = []
        for y2, x2 in ((y-1,x), (y,x-1), (y,x+1), (y+1,x)):
            unit = cave[y2, x2].unit
            if  unit != None and unit.type != self.type and unit.hp > 0:
                targets.append(unit)
        if len(targets) > 0:
            target = targets[np.argmin(np.array([t.hp for t in targets]))]
            target.hp -= self.atk
            if target.hp <= 0:
                target.tile.unit = None
        else:
            print('ERROR')
            print(index)
            prettyPrinter(cave)
            print([u.hp for u in units])
        
    def __str__(self):
        return self.type

class Tile:
    def __init__(self, char):
        if char in '#.':
            self.type  = char
            self.unit = None
        else:
            self.type = '.'
            self.unit = Unit(char, self)
            units.append(self.unit)
            
    def __str__(self):
        if self.unit != None:
            return str(self.unit)
        return self.type
    
    def isAvailable(self, unitType):        
        return self.type == '.' and (self.unit.type != unitType if self.unit != None else True)

def prettyPrinter(data):
    for row in data:
        print(''.join([str(col) for col in row]))
            
def bfs(grid, start, goals):
    width, height = grid.shape
    queue = collections.deque([[start]])
    seen  = set([start])
    unit  = cave[start].unit
    goalsWithPath = []
    lenShortestPath = 100000
    while queue:
        path = queue.popleft()
        y, x = path[-1]
        if grid[y][x].unit in goals:
            lenShortestPath = len(path) if len(path) < lenShortestPath else lenShortestPath
            goalsWithPath.append(path)
#             return path
        for y2, x2 in ((y-1,x), (y,x-1), (y,x+1), (y+1,x)):
            tile = grid[y2][x2]
            if 0 <= x2 < width and 0 <= y2 < height and tile.isAvailable(unit.type) and (y2, x2) not in seen:
                queue.append(path + [(y2, x2)])
                seen.add((y2, x2))
    
    shortestPaths = [x for x in goalsWithPath if len(x) == lenShortestPath]
    nearest = cave.shape
    #if len(shortestPaths) > 1:
    res = None
    for p in shortestPaths:
        y, x = p[-2]
        if y < nearest[0]:
            nearest = (y,x)
            res = p
        elif y == nearest[0]:
            if x < nearest[1]:
                nearest = (y,x)
                res = p
    return res

def move(tile, location):
    tile.unit.tile = cave[location]
    cave[location].unit = tile.unit
    tile.unit = None
    return cave[location]

In [325]:
units = []
cave = np.array([[Tile(char) for char in (line)] 
                 for line in (open('input.txt','r').read().splitlines())])
units = np.array(units)

iteration = 0
combat = True
while combat:
    for index, tile in np.ndenumerate(cave):
        if tile.unit != None and tile.unit.hp > 0 and not tile.unit.moved:
            targets = tile.unit.getTargets()
            if len(targets) > 0:
                path = bfs(cave, index, targets)
                if path == None:
                    continue                    
                elif len(path) == 2:
                    tile.unit.attack(index)
                elif len(path) == 3:
                    tile = move(tile, path[1])
                    tile.unit.attack(path[1])
                else:
                    tile = move(tile, path[1])
                tile.unit.moved = True
            else:
                combat = False
                print('Combat is over')
                finaliteration = iteration
                break
    for unit in units:
        unit.moved = False
    iteration += 1
    if iteration == 110:
        combat = False
        print('stopped')

sumhp = sum([u.hp for u in units if u.hp > 0])
print('Outcome:',finaliteration,'*',sumhp,'=',finaliteration*sumhp)
print('left:', sum([1 for u in units if u.type == 'E' and u.hp > 0]), '/', sum([1 for u in units if u.type == 'E']))
print(sumhp*44)
prettyPrinter(cave)





Combat is over
Outcome: 43 * 1187 = 51041
left: 10 / 10
52228
################################
##########..........############
########............E......#####
#######...............E.....####
#######............#....E.######
########................E..#...#
#######................E.#.....#
########................E......#
########...........#.....##....#
########.....#..............####
#########..........##.......#.##
##########..............#####.##
##########....#####.....####..##
######....EE.#######.....#.....#
###....#....#########......#####
####......E.#########......#####
###.......E.#########......#####
####........#########......#####
####..#.....#########....#######
######.......#######.....#######
###...........#####.....########
#......................#########
#......#..#..####....#.#########
#...#.........###.#..###########
##............###..#############
######......####..##############
######...........###############
#######.........################
######...####.